# Docs walkthrough
[*v0.1*]

Reference notebook while working through docs

# Setup

Cells in this section handle notebook setup, like importing packages and functions/vars from scripts.

## Imports

Import `stdlib` packages (i.e. `pathlib.Path`) and package dependencies.

### stdlib

In [ ]:
from pathlib import Path
import json
from typing import Any, Optional, Union, Optional, Type
from datetime import datetime, timedelta

### Dependencies

Packages installed with `pip` (or some equivalent tool)

In [ ]:
import sqlalchemy as sa
from sqlalchemy import ForeignKey, String, create_engine, MetaData, func
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship, Session

In [ ]:
from core.config import notebook_settings

display(f"Notebook settings: {notebook_settings}")

## Global Vars

Variables for use throughout the notebook

In [ ]:
nb_log: bool = notebook_settings.NB_LOG | False
nb_verbose: bool = notebook_settings.NB_VERBOSE | False

In [ ]:
## SQLAlchemy type mapping
#  https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html#mapped-column-derives-the-datatype-and-nullability-from-the-mapped-annotation
from sqlalchemy import types
import datetime
import decimal
import uuid

type_map: dict[Type[Any], types.TypeEngine[Any]] = {
    bool: types.Boolean(),
    bytes: types.LargeBinary(),
    datetime.date: types.Date(),
    datetime.datetime: types.DateTime(),
    datetime.timedelta: types.Interval(),
    decimal.Decimal: types.Numeric(),
    float: types.Float(),
    int: types.Integer(),
    str: types.String().with_variant(types.NVARCHAR, "mssql"),
    uuid.UUID: types.Uuid().with_variant(types.String, "sqlite"),
}

In [ ]:
## Map multiple type configurations to Python types
#  https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html#mapping-multiple-type-configurations-to-python-types
from typing_extensions import Annotated
from sqlalchemy import Numeric, String
from sqlalchemy.orm import registry

## Typing annotations here allow for type hints, i.e. in VSCode or Jupyter notebook
str_30: str = Annotated[str, 30]
str_50: str = Annotated[str, 50]
num_12_4: decimal.Decimal = Annotated[decimal.Decimal, 12]
num_6_2: decimal.Decimal = Annotated[decimal.Decimal, 6]

registry = registry(
    type_annotation_map={
        str_30: String(30),
        str_50: String(50),
        num_12_4: Numeric(12, 4),
        num_6_2: Numeric(6, 2),
    }
)

## Functions

Notebook-level functions. These differ from functions imported from scripts in that they are either prototypes, or functions meant only for the notebook.

### Notebook Functions

### Prototype

## Classes

Notebook-level classes. These differ from classes/models imported from scripts in that they are either prototypes, or functions meant only for the notebook.

In [ ]:
## SQLAlchemy declarative Base
#  https://docs.sqlalchemy.org/en/20/orm/declarative_styles.html#using-a-declarative-base-class
#  https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html#customizing-the-type-map
class Base(DeclarativeBase):
    type_annotation_map = type_map

In [ ]:
class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    nickname: Mapped[Optional[str]] = mapped_column(String(64))
    create_date: Mapped[datetime.datetime] = mapped_column(insert_default=func.now())

    addresses: Mapped["Address"] = relationship(back_populates="user")

    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r}, nickname={self.nickname!r})"

In [ ]:
class Address(Base):
    __tablename__ = "address"

    id: Mapped[int] = mapped_column(primary_key=True)
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    email_address: Mapped[str]

    user: Mapped["User"] = relationship(back_populates="addresses")

    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [ ]:
## Create user with dialect specified
#  https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html#customizing-the-type-map
from sqlalchemy.schema import CreateTable
from sqlalchemy.dialects import mssql, postgresql, sqlite

print(CreateTable(User.__table__).compile(dialect=sqlite.dialect()))

## SQLAlchemy connection & engine

In [ ]:
sqlite_db: str = "demo.sqlite"

SQLALCHEMY_DB_URI: str = f"sqlite:///{sqlite_db}"

In [ ]:
## Create engine
engine = create_engine(SQLALCHEMY_DB_URI, echo=True)

In [ ]:
SessionLocal: Session = Session(bind=engine)

# Operations

Functions & data operations.

In [ ]:
## Create metadata
Base.metadata.create_all(engine)

In [ ]:
## Create objects and persis
with SessionLocal as session:
    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )

    sandy = User(
        name="Sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )

    patrick = User(name="patrick", fullname="Patrick Star")

    ## Add objects to session and commit
    session.add_all([spongebob, sandy, patrick])

    session.commit()

In [ ]:
## Simple select

## Examples